# Crash Report Jan 18, 2016

Version: 2016070012

In [1]:
"""
General import statements
"""
import crash_analysis
import pandas as pd
from matplotlib import pyplot as plt 
%matplotlib inline

In [9]:
"""
Parse zipfiles into dataframe
"""
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe
zipfile_location = 'C:\\CrashReports\\'

extract_zipfiles(zipfile_location)
df = xmldocs_to_dataframe(zipfile_location)

df.drop_duplicates(inplace=True)

In [11]:
"""
Save/load dataframe to/from CSV
"""
from crash_analysis import read_csv

new_data = True
cache_location = 'src/data/2016070012_jan23_5pm.csv'

if new_data:
    df.to_csv(cache_location, encoding='utf-8')
else:
    df = read_csv(cache_location)

In [12]:
"""
Get metadata about dataframe
"""
col_names = crash_analysis.get_columns(df)
versions = df['AppVersion'].value_counts()
num_crashes = len(df)

print(col_names)
print(versions)
print(num_crashes)

['Unnamed: 0', 'ACCDT_Field', 'Active_ClientFileName', 'Active_Field', 'Active_Form', 'Active_FormsetID', 'Active_FormsetVersion', 'AppName', 'AppVersion', 'BasWin16.INI', 'Batch_ClientFileName', 'CrashGUID', 'CrashRpt', 'Current_Calcsection', 'CustNum', 'CustomProps', 'DataFileCount', 'ExceptionAddress', 'ExceptionCode', 'ExceptionModule', 'ExceptionModuleBase', 'ExceptionModuleVersion', 'ExceptionType', 'FileList', 'FormsPrinter', 'GUIResourceCount', 'GeoLocation', 'ImageName', 'InnerException', 'InstallType', 'InvParamExpression', 'InvParamFile', 'InvParamFunction', 'InvParamLine', 'Last_Calcsection', 'ManagedException', 'ManagedException.txt', 'MemoryUsageKbytes', 'Message', 'OSIs64Bit', 'OpenHandleCount', 'OperatingSystem', 'ProWin16.INI', 'ProblemDescription', 'Source', 'StackTrace', 'SystemTimeUTC', 'TimeStamp', 'WorkStationName', 'WorkStationType', 'crashdump.dmp', 'crashrpt.xml', 'empty']
2016070012    672
Name: AppVersion, dtype: int64
672


In [13]:
"""
Customer Description Analysis
"""
from crash_analysis.analysis import remove_empty, get_column, stem_frequency
customer_desc = remove_empty(get_column(df, 'ProblemDescription'))

stem_frequency(customer_desc)
print

total words: 723
tri        	:   34 	 [['trying'], ['trying'], ['try'], ['trying'], ['tried'], ['trying']]
program    	:   33 	 [['program'], ['program'], ['program'], ['program'], ['program'], ['program']]
file       	:   32 	 [['file'], ['file'], ['filing'], ['file'], ['file'], ['filing']]
open       	:   28 	 [['opening'], ['opening'], ['opened'], ['opening'], ['open'], ['open']]
save       	:   24 	 [['saving'], ['saving'], ['save'], ['save'], ['saving'], ['save']]
client     	:   24 	 [['client'], ['clients'], ['client'], ['client'], ['client'], ['client']]
print      	:   22 	 [['printing'], ['printing'], ['printing'], ['print'], ['print'], ['printing']]
return     	:   20 	 [['return'], ['returns'], ['returns'], ['returns'], ['return'], ['return']]
crash      	:   14 	 [['crashing'], ['crashes'], ['crashes'], ['crashing'], ['crashes'], ['crashes']]
year       	:   14 	 [['year'], ['years'], ['year'], ['years'], ['year'], ['year']]
softwar    	:   12 	 [['software'], ['software']

In [67]:
"""
Frequency of error messages and stack traces
"""

from crash_analysis import filter_dataframe


def get_value_from_hist(n, field='Message'):
    field_hist = df[field].value_counts()
    assert n < len(field_hist)
    return field_hist.keys()[n], field_hist.values[n]


def print_parings(n, field1='Message', groupings=['AppName', 'InstallType', 'OperatingSystem']):
    top_key, count = get_value_from_hist(n, field1)
    df_filtered = filter_dataframe(df, **{field1: top_key})
    if type(groupings) is type(list()):
        field2_hist = df_filtered.groupby(groupings)['CustNum'].nunique()
    else:
        field2_hist = df_filtered[groupings].value_counts()

    print("Query {0} \t count: {1} \t {2}: {3}".format(n+1, count, field1, top_key))
    print(field2_hist)
    print(sum(field2_hist))
    print '\n'

pd.set_option('display.max_rows', 5000)
pd.set_option('display.height', 5000)
# [print_parings(n, groupings='ProblemDescription') for n in range(5)]
# print_parings(0, groupings='CustNum')
# [print_parings(n, field1='ExceptionModuleBase', groupings='ExceptionAddress') for n in range(1,7)]
[print_parings(n, field1='StackTrace') for n in [0]]

# df[df['CustNum'] == '0000999480']['CrashGUID']

height has been deprecated.

Query 1 	 count: 74 	 StackTrace:    at _WinMainCRTStartup()
AppName                         InstallType  OperatingSystem                             
ProSeries - 2016                Network      Windows 10 Home Build 14393                     4
                                             Windows 10 Pro Build 14393                      4
                                             Windows 7 Home Premium Build 7601               1
                                             Windows 7 Professional Build 7601               4
                                             Windows 8 Pro Build 9200                        1
                                             Windows 8.1 Pro Build 9600                      1
                                             Windows Server 2008 R2 Standard Build 7601      2
                                             Windows Server 2012 R2 Datacenter Build 9600    2
                                             Windows Server 

[None]

In [79]:
""" Customer Descriptions """

from crash_analysis import remove_empty
# print(one_cust.head())
pd.set_option('display.max_colwidth', 250)
print(sum(remove_empty(df['ProblemDescription']).value_counts()))

35


In [80]:
"""
Associate by keyterm
"""
from crash_analysis.analysis import associate_by_keyterms

associate_by_keyterms(df, 'ProblemDescription', 'Message', min_count=2)
print

Message by Keyterm
keyterm: program
Method not found: 'Void Intuit.Application.ProSeries.OptionSetupWizardAPI.DataModels.AdditionalInfo..ctor(System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String)'.    2
Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.                                                                                                                                                                                                                      1
External component has thrown an exception.                                                                                                                                                                                                                                                        1
Name: Message, dtype: int64

keyterm: open
External component has thrown an exception